# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/home/travis/miniconda/envs/test/lib/python3.7/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.20.0.45/12437/1 Dashboard: http://10.20.0.45:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1017,Edith,-0.6273097372319765,0.7239174698486881
2000-01-01 00:00:01,971,Quinn,-0.15444030858879132,-0.18649992104150792
2000-01-01 00:00:02,1014,Sarah,0.1530846764741891,-0.07341493608423066
2000-01-01 00:00:03,972,Hannah,0.13611247467923882,0.9791035382452951
2000-01-01 00:00:04,1007,Oliver,-0.03674906235959674,-0.35175550487753915
2000-01-01 00:00:05,1011,Yvonne,0.22125205928273095,-0.9410038215132326
2000-01-01 00:00:06,997,Quinn,0.5317860263582561,0.9565202362787315
2000-01-01 00:00:07,1040,Edith,0.7030667755041595,-0.3280878898301729
2000-01-01 00:00:08,996,Sarah,0.8767987619552047,0.7365383695944097


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,989,Yvonne,0.5811536598775144,0.9007155628312025
2000-01-30 00:00:01,1058,Sarah,-0.04316876304249817,-0.24261704150473395
2000-01-30 00:00:02,1021,Patricia,0.7433270497534792,0.26874719798106206
2000-01-30 00:00:03,978,Patricia,-0.9856445719636413,0.3772314056488357
2000-01-30 00:00:04,991,Ingrid,-0.15855182078594998,-0.19626745921917843
2000-01-30 00:00:05,996,Sarah,0.33486472677875434,-0.7267744359994186
2000-01-30 00:00:06,997,Kevin,0.9770778110172684,-0.9414955451284621
2000-01-30 00:00:07,1002,Sarah,0.7269062158703401,-0.9652221961314098
2000-01-30 00:00:08,999,Ray,0.8382214968069768,-0.11387723767222169


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1017,Edith,-0.627310,0.723917
1,2000-01-01 00:00:01,971,Quinn,-0.154440,-0.186500
2,2000-01-01 00:00:02,1014,Sarah,0.153085,-0.073415
3,2000-01-01 00:00:03,972,Hannah,0.136112,0.979104
4,2000-01-01 00:00:04,1007,Oliver,-0.036749,-0.351756


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1017,Edith,-0.627310,0.723917
1,2000-01-01 00:00:01,971,Quinn,-0.154440,-0.186500
2,2000-01-01 00:00:02,1014,Sarah,0.153085,-0.073415
3,2000-01-01 00:00:03,972,Hannah,0.136112,0.979104
4,2000-01-01 00:00:04,1007,Oliver,-0.036749,-0.351756


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 4.68 s, sys: 635 ms, total: 5.32 s
Wall time: 3.46 s


name
Alice       0.000227
Bob        -0.001511
Charlie     0.001825
Dan         0.000366
Edith       0.001100
Frank      -0.001347
George     -0.003009
Hannah      0.003361
Ingrid     -0.001091
Jerry       0.000695
Kevin       0.001133
Laura      -0.001787
Michael    -0.003017
Norbert    -0.001463
Oliver      0.000878
Patricia    0.000996
Quinn       0.004838
Ray        -0.001329
Sarah      -0.002212
Tim        -0.000918
Ursula      0.001488
Victor      0.000274
Wendy      -0.003239
Xavier     -0.002230
Yvonne      0.001022
Zelda       0.000880
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.15.parquet  part.22.parquet  part.2.parquet
_metadata	  part.16.parquet  part.23.parquet  part.3.parquet
part.0.parquet	  part.17.parquet  part.24.parquet  part.4.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.5.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.6.parquet
part.12.parquet   part.1.parquet   part.27.parquet  part.7.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.8.parquet
part.14.parquet   part.21.parquet  part.29.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.51 s, sys: 133 ms, total: 1.64 s
Wall time: 1.11 s


name
Alice       0.000227
Bob        -0.001511
Charlie     0.001825
Dan         0.000366
Edith       0.001100
Frank      -0.001347
George     -0.003009
Hannah      0.003361
Ingrid     -0.001091
Jerry       0.000695
Kevin       0.001133
Laura      -0.001787
Michael    -0.003017
Norbert    -0.001463
Oliver      0.000878
Patricia    0.000996
Quinn       0.004838
Ray        -0.001329
Sarah      -0.002212
Tim        -0.000918
Ursula      0.001488
Victor      0.000274
Wendy      -0.003239
Xavier     -0.002230
Yvonne      0.001022
Zelda       0.000880
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.35 s, sys: 79.7 ms, total: 1.43 s
Wall time: 995 ms


name
Alice       0.000227
Bob        -0.001511
Charlie     0.001825
Dan         0.000366
Edith       0.001100
Frank      -0.001347
George     -0.003009
Hannah      0.003361
Ingrid     -0.001091
Jerry       0.000695
Kevin       0.001133
Laura      -0.001787
Michael    -0.003017
Norbert    -0.001463
Oliver      0.000878
Patricia    0.000996
Quinn       0.004838
Ray        -0.001329
Sarah      -0.002212
Tim        -0.000918
Ursula      0.001488
Victor      0.000274
Wendy      -0.003239
Xavier     -0.002230
Yvonne      0.001022
Zelda       0.000880
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html